# AE Autoencoders
### From scratch

## Imports

In [1]:
import numpy as np
import pandas as pd 
import math 
import tensorflow as tf 

In [7]:
class AE(object):

    # Fields
    nHidden = 2
    epochs = 1000
    batchSize = 10
    Whid = None
    bhid = None
    Hout = None
    Wout = None
    bout = None
    
    Yout = None
    Yexp = None
    Out = None
    inputOrig = None
    input = None
    output = None
    nSamp = 0
    nVar = 3
    X = tf.placeholder("float", [None, nVar]) 
 
    
    
    # Constructor
    def __init__(self):
        return
      
    # Normalize to [0,1] 
    def Norm01(self, data):
        nData = np.divide((data-data.min()), (data.max() - data.min())) 
        return nData
    
    # Normalize to [-1,1] 
    def Norm_11(self, data):
        nData = np.divide((data-data.min()), (data.max() - data.min())) 
        n1Data = (nData*2)-1 
        return n1Data

    def Build(self, input): 
         
        # Normalization (in this case [-1,1]
        self.inputOrig = input
        output = input 
        self.input = self.Norm_11(input) 
        self.output = self.Norm_11(output) 
         
        self.nSamp, self.nVar = self.input.shape  
        self.batchSize = min(self.batchSize, self.nSamp) 
        
        # input layer
        self.X = tf.placeholder("float", [None, self.nVar]) 
       
        # hidden layer 
        self.Whid = tf.Variable(tf.random_uniform((self.nVar, self.nHidden), -1.0 / math.sqrt(self.nVar), 1.0 / math.sqrt(self.nVar))) 
        self.bhid = tf.Variable(tf.zeros([self.nHidden])) 
        self.Yhid = tf.nn.tanh(tf.matmul(self.X,self.Whid) + self.bhid) 
        
        # output layer
        self.Wout = tf.transpose(self.Whid) # tied weights 
        self.bout = tf.Variable(tf.zeros([self.nVar])) 
        self.Yout = tf.nn.tanh(tf.matmul(self.Yhid,self.Wout) + self.bout) 
        
        # expected outcome
        self.Yexp = tf.placeholder("float", [None,self.nVar]) 
        return
    
    def Train(self, log=False): 
        
        mse = tf.reduce_mean(tf.square(self.Yout - self.Yexp)) #crossEntropy = -tf.reduce_sum(self.Yexp * tf.log(self.Yout)) 
        trainStep = tf.train.GradientDescentOptimizer(learning_rate=0.05).minimize(mse) 
        
        init = tf.global_variables_initializer()
        sess = tf.Session() 
        sess.run(init) 
   
        for i in range(self.epochs): 
            sample = np.random.randint(self.nSamp, size=self.batchSize) 
            bX = self.input[sample][:] 
            bY = self.output[sample][:] 
            sess.run(trainStep, feed_dict={self.X: bX, self.Yexp:bY}) 
            if i % 100 == 0 and log==True: 
                Mse = sess.run(mse, feed_dict={self.X: bX, self.Yexp:bY}); print(i, " : ", round(Mse,5)) 
        self.Out = sess.run(self.Yout, feed_dict={self.X: self.input})
        return
    
    def Output(self):
        #print("Expected:"); print(self.output) 
        #print( "Output:"); print(np.round(self.Out, 2)); 
        diff = np.square(self.output - self.Out); 
        M = np.mean(pd.DataFrame(diff), axis=1); 
        Mm = M.as_matrix().reshape(M.shape[0], 1)
        res = np.append(self.inputOrig, Mm, axis=1); print(res)
        return res

## Testing

In [8]:
#np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
        
print ("Test AE")

input = np.array([[2.0, 1.0, 1.0, 2.0], [-2.0, 1.0, -1.0, 2.0], [0.0, 1.0, 0.0, 2.0], [0.0, -1.0, 0.0, -2.0], [0.0, -1.0, 0.0, -2.0]]) 
print(input) 

ae = AE()
ae.Build(input)
ae.Train(True)
res = ae.Output()
print(res)

Test AE
[[2.000 1.000 1.000 2.000]
 [-2.000 1.000 -1.000 2.000]
 [0.000 1.000 0.000 2.000]
 [0.000 -1.000 0.000 -2.000]
 [0.000 -1.000 0.000 -2.000]]
0  :  0.30752
100  :  0.08963
200  :  0.02
300  :  0.00785
400  :  0.00454
500  :  0.00478
600  :  0.00319
700  :  0.00256
800  :  0.00328
900  :  0.00267
[[2.000 1.000 1.000 2.000 0.004]
 [-2.000 1.000 -1.000 2.000 0.004]
 [0.000 1.000 0.000 2.000 0.001]
 [0.000 -1.000 0.000 -2.000 0.002]
 [0.000 -1.000 0.000 -2.000 0.002]]
[[2.000 1.000 1.000 2.000 0.004]
 [-2.000 1.000 -1.000 2.000 0.004]
 [0.000 1.000 0.000 2.000 0.001]
 [0.000 -1.000 0.000 -2.000 0.002]
 [0.000 -1.000 0.000 -2.000 0.002]]


c:\program files\python36\lib\site-packages\ipykernel_launcher.py:92: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [ ]:
import numpy as np
import AE as Ae
import MNIST_data.inputData as id


# Test TsCat, transforming numerical time series in categorical
# -------------------------------------------------------------------
   
class AETest(unittest.TestCase):

    # Setup & TearDown
    # ---------------------------------------------------------------
    
    def setUp(self):
        pass


    def tearDown(self):
        pass

    if __name__ == "__main__":
        #import sys;sys.argv = ['', 'Test.testName']
        unittest.main()
        
 # Tests AE: Test BinVar and BinTS
 # ---------------------------------------------------------------
    
    def testAE(self):
        np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
        
        print ("Test AE")
        
        input = np.array([[2.0, 1.0, 1.0, 2.0], [-2.0, 1.0, -1.0, 2.0], [0.0, 1.0, 0.0, 2.0], [0.0, -1.0, 0.0, -2.0], [0.0, -1.0, 0.0, -2.0]]) ; print(input) 
        ae = Ae.AE()
        ae.Build(input)
        ae.Train(True)
        res = ae.Output()
        print(res)
        pass

    def train(self, nnArch, mnist=None, lrate=0.001, bSize=100, epochs=10, printStep=5, nSamples=100):
    
        vae = Vae.VAE(nnArch, lrate=lrate, bSize=bSize)
        
        for epoch in range(epochs):
            avg_cost = 0.
            total_batch = int(nSamples / bSize)
            for i in range(total_batch):
                batch_xs, _ = mnist.train.next_batch(bSize)
                cost = vae.Fit(batch_xs)
                avg_cost += cost[1] / nSamples * bSize
                if epoch % printStep == 0:
                    print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))

## Credits & links

https://stackabuse.com/autoencoders-for-image-reconstruction-in-python-and-keras/